In [1]:
from src.mcq_generation import *

In [11]:
table_name = 'mcq_metadata'
database_file = "../database/mcq_metadata.db"

In [4]:
table_exists(table_name, database_file)

True

In [4]:
from src.database_query import *

In [5]:
export_table_to_csv("mcq_metadata", "20250414_mcq_metadata.csv")

Exported 7 rows to ../output\20250414_mcq_metadata.csv


In [5]:
# Example usage of the function
extraction_list = get_extraction_values()

In [6]:
extraction_list

['What is the likely outcome when organisms fail to capture and consume all available food at a trophic level, as mentioned in the text?\nA) Populations at higher trophic levels increase rapidly.\nB) Energy transfer from one trophic level to the next is more efficient.\nC) The total energy available to organisms decreases at each successive trophic level.\nD) Ecologists find it challenging to construct ecological pyramids accurately.',
 'A) Decrease in biodiversity\nB) Increase in energy transfer efficiency\nC) Higher population sizes at higher trophic levels\nD) Release of energy as heat into the environment',
 'Based on the provided text, why do organisms fail to capture and eat all the food available at the trophic level below them in an ecosystem?\nA) Lack of appetite\nB) Inefficiency in digestion\nC) Overpopulation at lower trophic levels\nD) Excessive energy consumption\nE) Environmental changes affecting food availability']

In [1]:
from src.mcq_generation import generate_mcq
from src.general import *

In [2]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")
text = read_text_file(files_to_process[0])

In [3]:
text

'Energy and trophic levels: Ecological pyramids How can you show how energy is used in an ecosystem? Ecologists use food chains and food webs to model the distribution of matter and energy within an ecosystem. They also use another kind of model, called an ecological pyramid. An ecological pyramid shows how energy flows through an ecosystem. The base of the ecological pyramid represents the autotrophs, or first trophic level. Higher trophic levels are layered on top of one another. The initial source of energy for all ecological pyramids is energy from the sun. Energy decreases at each succeeding trophic level. The total energy transfer from one trophic level to the next is only about ten percent because organisms fail to capture and eat all the food available at the trophic level below them. When an organism consumes food, it uses some of the energy in the food for metabolism, some for building body tissues, and some is given off as waste. When the organism is eaten, the energy that w

In [5]:
table_name = 'mcq_metadata'
database_file = "../database/mcq_metadata.db"

In [6]:
generate_mcq(text, question_type="inference", table_name=table_name, database_file=database_file)

{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYou are asked to write an inferential multiple-choice question based on an academic text provided by the user. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are inferential questions?**\nInferential questions require students to go beyond the explicit information in a text and draw conclusions based on evidence, reasoning, and prior knowledge. Unlike literal questions, which ask for directly stated facts, inferential questions challenge students to interpret, analyze, or predict based on textual clues.\nFor example, \nExcerpt:\n"Studies have shown that prolonged exposure to stress can negatively impact cognitive function, particularly memory and decision-making. This is because chronic s

In [5]:
from src.prompt_fetch import *
from src.agent import Agent
from src.general import *

In [6]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")


In [7]:
files_to_process

['../data/text1.txt']

In [9]:
# get prompts 
prompts = get_prompts("inference_prompts.yaml")

In [10]:
prompts

{'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYou are asked to write an inferential multiple-choice question based on an academic text provided by the user. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are inferential questions?**\nInferential questions require students to go beyond the explicit information in a text and draw conclusions based on evidence, reasoning, and prior knowledge. Unlike literal questions, which ask for directly stated facts, inferential questions challenge students to interpret, analyze, or predict based on textual clues.\nFor example, \nExcerpt:\n"Studies have shown that prolonged exposure to stress can negatively impact cognitive function, particularly memory and decision-making. This is because chronic s

In [11]:
text = read_text_file(files_to_process[0])

In [12]:
text

'Energy and trophic levels: Ecological pyramids How can you show how energy is used in an ecosystem? Ecologists use food chains and food webs to model the distribution of matter and energy within an ecosystem. They also use another kind of model, called an ecological pyramid. An ecological pyramid shows how energy flows through an ecosystem. The base of the ecological pyramid represents the autotrophs, or first trophic level. Higher trophic levels are layered on top of one another. The initial source of energy for all ecological pyramids is energy from the sun. Energy decreases at each succeeding trophic level. The total energy transfer from one trophic level to the next is only about ten percent because organisms fail to capture and eat all the food available at the trophic level below them. When an organism consumes food, it uses some of the energy in the food for metabolism, some for building body tissues, and some is given off as waste. When the organism is eaten, the energy that w

In [14]:
model = "gpt-3.5-turbo"
system_prompt = prompts["system_prompt"]
user_prompt = prompts["user_prompt"].format(text=text)

In [15]:
test_agent = Agent(model=model, system_prompt=system_prompt, user_prompt=user_prompt)

In [16]:
test_agent.completion_generation()

'**Inferential Multiple-Choice Question:** \nWhat is the likely effect of organisms failing to capture and eat all the food available at a trophic level, as described in the text?\n\nA) It leads to an increase in biomass at higher trophic levels.\nB) It results in a decrease in population sizes at higher trophic levels.\nC) It causes a surplus of available energy at lower trophic levels.\nD) It contributes to a redistribution of energy as heat into the environment.\n\nAnswer: D) It contributes to a redistribution of energy as heat into the environment.'

In [13]:
test_agent.get_metadata()

{'task': None,
 'system_prompt': '[ROLE]:\nYou are an experienced college instructor. You are an expert in writing multiple-choice questions to assess students’ understanding of academic texts (e.g., chapters in textbooks, academic articles).\n\n[TASK]:\nYou are asked to write an inferential multiple-choice question based on an academic text provided by the user. \nFollow the guidelines in [GUIDELINES] to do exactly what you are instructed to do. \n\n[DEFINITION]\n**What are inferential questions?**\nInferential questions require students to go beyond the explicit information in a text and draw conclusions based on evidence, reasoning, and prior knowledge. Unlike literal questions, which ask for directly stated facts, inferential questions challenge students to interpret, analyze, or predict based on textual clues.\nFor example, \nExcerpt:\n"Studies have shown that prolonged exposure to stress can negatively impact cognitive function, particularly memory and decision-making. This is be

In [1]:
80000000/1057

75685.90350047304

In [1]:
from src.text_processing import *

en_core_web_sm is already installed.


In [5]:
from src.general import *

# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")
text = read_text_file(files_to_process[1])

In [6]:
text

'The Alaska pipeline starts at the frozen edge of the Arctic Ocean. It stretches southward across the largest and northernmost state in the United States, ending at a remote ice-free seaport village nearly 800 miles from where it begins. It is massive in size and extremely complicated to operate.\nThe steel pipe crosses windswept plains and endless miles of delicate tundra that tops the frozen ground. It weaves through crooked canyons, climbs sheer mountains, plunges over rocky crags, makes its way through thick forests, and passes over or under hundreds of rivers and streams. The pipe is 4 feet in diameter, and up to 2 million barrels (or 84 million gallons) of crude oil can be pumped through it daily. Resting on H-shaped steel racks called "bents," long sections of the pipeline follow a zigzag course high above the frozen earth. Other long sections drop out of sight beneath spongy or rocky ground and return to the surface later on. The pattern of the pipeline\'s up-and-down route is 

In [7]:
add_chunk_markers(text)

'<chunk1>The Alaska pipeline starts at the frozen edge of the Arctic Ocean. It stretches southward across the largest and northernmost state in the United States, ending at a remote ice-free seaport village nearly 800 miles from where it begins. It is massive in size and extremely complicated to operate.\nThe steel pipe crosses windswept plains and endless miles of delicate tundra that tops the frozen ground. It weaves through crooked canyons, climbs sheer mountains, plunges over rocky crags, makes its way through thick forests, and passes over or under hundreds of rivers and streams. The pipe is 4 feet in diameter, and up to 2 million barrels (or 84 million gallons) of crude oil can be pumped through it daily. Resting on H-shaped steel racks called "bents," long sections of the pipeline follow a zigzag course high above the frozen earth. Other long sections drop out of sight beneath spongy or rocky ground and return to the surface later on. The pattern of the pipeline\'s up-and-down r

## Test Preprocessor

In [1]:
from src.text_processing import *

en_core_web_sm is already installed.


In [2]:
from src.general import *

In [3]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")
text = read_text_file(files_to_process[2])

In [4]:
text

'Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008\n\nThe sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan.\n\nâ€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develop cancer in their internal organs. Moreover, they show that cancer patients have a better prognosis if they are diagnosed in summer or autumn, when

In [5]:
chunked_text = add_chunk_markers(text)

In [6]:
chunked_text

'<chunk1>Research magazine from Stanford University Journalist Anne Simonson â€“ January 13, 2008 The sun counteracts cancer\nSunrays that hit our skin reinforce the bodyâ€™s own defense against cancer. The campaigns to get people out of the sun have gone too far, according to the researcher Johan Moan. â€œThe rays of the sun speed up the bodyâ€™s production of vitamin D. This can prevent cancer as well as many other diseases.â€\x9d So says Johan Moan, Professor at the Cancer Center at Boston Medical Center. Along with colleagues from Norway, he is presenting results that support the theory that sun exposure provides the body with a more effective defense against cancer. The findings show, for example, that people who live closer to the equator, and whose bodies hence are more exposed to the sun, have a better chance of survival if they develop cancer in their internal organs. Moreover, they show that cancer patients have a better prognosis if they are diagnosed in summer or autumn, wh

## Test Planner

In [7]:
from src.planner import *

In [8]:
invocation_id = "rstikkwls"


In [9]:
plan = generate_plan(
    invocation_id=invocation_id,
    model="gpt-4o",
    text=chunked_text, 
    fact = 1, 
    inference = 1, 
    table_name = "plan_metadata", 
    database_file = "../database/mcq_metadata.db")

2025-05-20 17:18:02,819 - INFO - Table 'plan_metadata' created or already exists.
2025-05-20 17:18:22,737 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-20 17:18:22,744 - INFO - Plan generated successfully.
2025-05-20 17:18:22,745 - INFO - Generated text converted to dictionary successfully.
2025-05-20 17:18:22,766 - INFO - Metadata inserted into 'plan_metadata'.


In [10]:
plan

{'system_prompt': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/or inferences\n  Depending on the user’s requests, create a list of key facts and/or inferences from the academic text. The number of facts or inferences can be more than what the user requests. \n  Skip 

## Test Controller

In [11]:
from src.controller_helper import *

In [12]:
fact =1
inference = 1
main_idea = 1

In [13]:
task_list = create_task_list(plan, fact, inference, main_idea)

In [14]:
task_list

[{'question_type': 'fact',
  'content': "Sun exposure boosts the body's production of vitamin D, which can prevent cancer and other diseases.",
  'text': '',
  'context': "Research from Stanford University's magazine discusses how sunrays that hit our skin bolster the body's defense against cancer. Researcher Johan Moan argues that sun exposure, which increases vitamin D production, can prevent cancer and other diseases. Those living closer to the equator have better cancer survival rates, and patients have improved prognoses if diagnosed in sunnier months. This research has been published in the Proceedings of the National Academy of Sciences. Avoiding sun exposure may be harmful, especially for northerners who are sun-deprived, notes collaborator Richard Setlow. Many are advised to avoid sun exposure, but Moan suggests minimal sunscreen-free sun exposure is healthy. Half an hour outdoors in midsummer provides vitamin D equivalent to a small bottle of cod-liver oil. During low-sun mon